In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory


In [ ]:
# Reforestation Optimization Model: Multi-Stage (Pyomo)


# ------------------
# DATA STRUCTURES
# ------------------

# Example data placeholders - replace with actual inputs
species = ['S1', 'S2', 'S3']
nurseries = ['N1', 'N2', 'N3', 'N4']
polygons = list(range(1, 32))
days = list(range(1, 31))
hours_per_day = {1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 3}  # Day of week
plant_costs = {'N1': {'S1': 2.5}, 'N2': {'S2': 3.0}}  # $/plant
survival_rate_matrix = {(s, d): 0.8 + 0.01 * d for s in species for d in range(3, 8)}
preprocessing_time = {'S1': 20, 'S2': 60, 'S3': 40}  # in minutes

# Travel time/distance matrix in minutes and meters (Polygon 18 is the storage site)
distance_matrix = {(18, p): 1000 for p in polygons}
distance_time_matrix = {(18, p): distance_matrix[(18, p)] / 100.0 for p in polygons}  # 1 min per 100m

# Demands: (polygon, species): quantity
polygon_demands = {(p, s): 100 for p in polygons for s in species}
nursery_species = {'N1': ['S1'], 'N2': ['S2'], 'N3': ['S3'], 'N4': ['S1', 'S3']}

# ------------------
# MODEL
# ------------------
model = ConcreteModel()

# Sets
model.DAYS = Set(initialize=days)
model.SPECIES = Set(initialize=species)
model.NURSERIES = Set(initialize=nurseries)
model.POLYGONS = Set(initialize=polygons)

# Parameters
model.TruckCapacity = Param(initialize=8000)
model.TruckCost = Param(initialize=4500)
model.StorageArea = Param(initialize=400)  # m^2
model.PreprocessingArea = Param(initialize=100)
model.PlantCost = Param(initialize=20)
model.UploadDownloadTime = Param(initialize=60)  # 30 min each

# Distance and Time Parameters
model.DistanceTime = Param(model.POLYGONS, initialize=lambda m, p: distance_time_matrix[(18, p)])

# Demand
model.Demand = Param(model.POLYGONS, model.SPECIES, initialize=lambda m, p, s: polygon_demands.get((p, s), 0))

# ------------------
# VARIABLES
# ------------------
# Orders
model.Orders = Var(model.NURSERIES, model.SPECIES, model.DAYS, domain=NonNegativeIntegers)

# Plants in storage by day
model.Storage = Var(model.SPECIES, model.DAYS, domain=NonNegativeIntegers)

# Preprocessing
model.Preprocessed = Var(model.SPECIES, model.DAYS, domain=NonNegativeIntegers)

# Dispatch to polygon
model.Dispatch = Var(model.SPECIES, model.POLYGONS, model.DAYS, domain=NonNegativeIntegers)

# Planted
model.Planted = Var(model.SPECIES, model.POLYGONS, model.DAYS, domain=NonNegativeIntegers)

# Truck usage
model.TruckUsed = Var(model.DAYS, domain=Binary)

# ------------------
# CONSTRAINTS
# ------------------

# Storage flow constraint
# Storage[t] = Storage[t-1] + Orders[t-1] - Dispatch[t] (simplified for demo)
def storage_flow_rule(m, s, d):
    if d == 1:
        return m.Storage[s, d] == 0
    return m.Storage[s, d] == m.Storage[s, d - 1] + sum(m.Orders[n, s, d - 1] for n in m.NURSERIES) - sum(m.Dispatch[s, p, d - 1] for p in m.POLYGONS)
model.StorageFlow = Constraint(model.SPECIES, model.DAYS, rule=storage_flow_rule)

# Storage area constraint (not exceed 400 m^2)
# Assume 1 plant = 0.01 m^2 for this example
model.StorageCapacity = Constraint(model.DAYS, rule=lambda m, d: sum(m.Storage[s, d] * 0.01 for s in m.SPECIES) <= m.StorageArea)

# Preprocessing constraint
model.PreprocessingLimit = Constraint(model.DAYS, rule=lambda m, d: sum(m.Preprocessed[s, d] * preprocessing_time[s] for s in m.SPECIES) <= hours_per_day.get((d - 1) % 7 + 1, 6) * 60)

# Truck route time + upload/download <= working hours
model.TruckTime = Constraint(model.DAYS, rule=lambda m, d: sum(m.TruckUsed[d] * (m.DistanceTime[p] + m.UploadDownloadTime) for p in m.POLYGONS) <= hours_per_day.get((d - 1) % 7 + 1, 6) * 60)

# Daily planting area constraint (>= 5 ha per day => 50000 m^2)
# Assume 1 plant = 1 m^2
model.PlantingMinimum = Constraint(model.DAYS, rule=lambda m, d: sum(m.Planted[s, p, d] for s in m.SPECIES for p in m.POLYGONS) >= 50000)

# ------------------
# OBJECTIVE FUNCTION
# ------------------
def objective_rule(m):
    cost = sum(m.Orders[n, s, d] * plant_costs.get(n, {}).get(s, 0) for n in m.NURSERIES for s in m.SPECIES for d in m.DAYS)
    transport = sum(m.TruckUsed[d] * m.TruckCost for d in m.DAYS)
    planting = sum(m.Planted[s, p, d] * m.PlantCost for s in m.SPECIES for p in m.POLYGONS for d in m.DAYS)
    return cost + transport + planting

model.TotalCost = Objective(rule=objective_rule, sense=minimize)

# ------------------
# SOLVER
# ------------------
# opt = SolverFactory('cbc')
# result = opt.solve(model, tee=True)
# model.display()

# ------------------
# NOTE:
# This is a starter template. You will need to:
# - Add linking constraints across stages
# - Load actual data matrices (distances, demand, survival, etc.)
# - Tune for large-scale feasibility (decomposition or heuristics)


In [1]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpInteger, LpStatus
import pandas as pd

# Data setup
species = [f"E{i}" for i in range(1, 11)]
vendors = [f"V{i}" for i in range(1, 5)]
days = [3, 4, 5, 6]

# Survival rates for each species on Day 6
survival_day6 = {
    "E1": 0.9, "E2": 0.92, "E3": 0.88, "E4": 0.87, "E5": 0.9,
    "E6": 0.91, "E7": 0.92, "E8": 0.91, "E9": 0.75, "E10": 0.85
}

# Cost matrix per species and vendor
cost_matrix = {
    "E1": [0, 0, 0, 26],
    "E2": [0, 0, 0, 26],
    "E3": [0, 26, 0, 26],
    "E4": [0, 26, 25, 0],
    "E5": [0, 17, 18, 0],
    "E6": [0, 0, 18, 21],
    "E7": [0, 17, 18, 18],
    "E8": [0, 0, 18, 0],
    "E9": [26.5, 0, 0, 0],
    "E10": [26, 0, 0, 0]
}

# Truck capacity in plants (199x127cm truck, 25x25cm plant)
truck_capacity = (199 // 25) * (127 // 25)  # 7x5 = 35 plants per trip

# Stage 1: Vendor Selection and Purchase Quantity
model = LpProblem("Minimize_Cost_Maximize_Survival", LpMinimize)

# Decision variables: number of each species from each vendor
x = LpVariable.dicts("PlantQty", ((s, v) for s in species for v in vendors), lowBound=0, cat=LpInteger)

# Objective: Minimize total cost
model += lpSum(x[(s, v)] * cost_matrix[s][vendors.index(v)] for s in species for v in vendors)

# Constraints: Only allow purchase if vendor offers that species
for s in species:
    for v in vendors:
        if cost_matrix[s][vendors.index(v)] == 0:
            model += x[(s, v)] == 0

# Constraint: Total number of plants cannot exceed maximum truck capacity per trip * 1 trip per day over 4 days
model += lpSum(x[(s, v)] for s in species for v in vendors) <= truck_capacity * len(days)

# Solve Stage 1
model.solve()

# Extract solution
plant_orders = {(s, v): int(x[(s, v)].varValue) for s in species for v in vendors if x[(s, v)].varValue > 0}

# Stage 2: Planting Scheduling
# Create schedule dictionary: {day: [(species, vendor, quantity)]}
schedule = {day: [] for day in days}
planting_plan = []

# Simple greedy allocation across 4 days, max 35 plants per day
remaining = sum(plant_orders.values())
order_list = [(s, v, q) for (s, v), q in plant_orders.items()]

day_index = 0
day_totals = {day: 0 for day in days}

for s, v, q in order_list:
    while q > 0:
        day = days[day_index % 4]
        available = truck_capacity - day_totals[day]
        assign = min(available, q)
        if assign > 0:
            schedule[day].append((s, v, assign))
            day_totals[day] += assign
            planting_plan.append((day, s, v, assign))
            q -= assign
        if day_totals[day] == truck_capacity:
            day_index += 1

# Stage 3: Compute expected survival after planting
total_expected_survivors = sum(assign * survival_day6[s] for day, s, v, assign in planting_plan)
total_cost = sum(qty * cost_matrix[s][vendors.index(v)] for (s, v), qty in plant_orders.items())

# Convert schedule to DataFrame for presentation
schedule_df = pd.DataFrame(planting_plan, columns=["Day", "Species", "Vendor", "Quantity"])
schedule_df.sort_values(by="Day", inplace=True)

plant_orders, total_cost, total_expected_survivors, schedule_df



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/samanthabritoozuna/anaconda3/envs/tecs6-arm/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/f96fdd7563424f92a6519ef03bb0ac99-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/f96fdd7563424f92a6519ef03bb0ac99-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 191 RHS
At line 217 BOUNDS
At line 258 ENDATA
Problem MODEL has 25 rows, 40 columns and 64 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 times and

({},
 0,
 0,
 Empty DataFrame
 Columns: [Day, Species, Vendor, Quantity]
 Index: [])

In [3]:
import pulp
import pandas as pd

# ------------------------
# 1. INPUT DATA STRUCTURES
# ------------------------

# Example data: should be replaced with real data from Excel, CSV, or a DB
plants = ['Plant1', 'Plant2', 'Plant3']
clients = ['ClientA', 'ClientB']
storage_locations = ['WH1']
trucks = ['Truck1', 'Truck2']

# Demand: [plant][client] = quantity requested
demand = {
    ('Plant1', 'ClientA'): 50,
    ('Plant2', 'ClientA'): 30,
    ('Plant1', 'ClientB'): 40,
}

# Inventory at storage: [plant][storage] = available stock
inventory = {
    ('Plant1', 'WH1'): 100,
    ('Plant2', 'WH1'): 50,
    ('Plant3', 'WH1'): 20,
}

# Truck capacities (in volume units)
truck_capacity = {
    'Truck1': 80,
    'Truck2': 100,
}

# Distance or cost matrix (for routing costs)
transport_cost = {
    ('WH1', 'ClientA'): 10,
    ('WH1', 'ClientB'): 20,
}

# ------------------------
# 2. DEFINE THE OPTIMIZATION MODEL
# ------------------------

model = pulp.LpProblem("SupplyChainOptimization", pulp.LpMinimize)

# ------------------------
# 3. DECISION VARIABLES
# ------------------------

# x[plant, client, truck] = quantity of plant delivered by truck to client
x = pulp.LpVariable.dicts("delivered",
                          ((p, c, t) for p in plants for c in clients for t in trucks),
                          lowBound=0, cat='Continuous')

# y[plant, storage] = amount of plant stored (shipped from storage)
y = pulp.LpVariable.dicts("from_storage",
                          ((p, s) for p in plants for s in storage_locations),
                          lowBound=0, cat='Continuous')

# z[client] = binary flag if demand is fulfilled (optional for partial delivery cases)
# z = pulp.LpVariable.dicts("fulfilled", clients, cat='Binary')

# ------------------------
# 4. OBJECTIVE FUNCTION
# ------------------------

# Example: minimize transport cost
model += pulp.lpSum(
    x[p, c, t] * transport_cost[('WH1', c)]
    for p in plants for c in clients for t in trucks if ('WH1', c) in transport_cost
)

# ------------------------
# 5. CONSTRAINTS
# ------------------------

# Demand fulfillment
for p, c in demand:
    model += pulp.lpSum(x[p, c, t] for t in trucks) >= demand[(p, c)], f"demand_{p}_{c}"

# Inventory constraint: can’t ship more than available
for p in plants:
    for s in storage_locations:
        total_shipped = pulp.lpSum(x[p, c, t] for c in clients for t in trucks)
        model += total_shipped <= inventory.get((p, s), 0), f"inventory_{p}_{s}"

# Truck capacity
for t in trucks:
    model += pulp.lpSum(x[p, c, t] for p in plants for c in clients) <= truck_capacity[t], f"capacity_{t}"

# ------------------------
# 6. SOLVE THE MODEL
# ------------------------

model.solve()
print("Status:", pulp.LpStatus[model.status])

# ------------------------
# 7. OUTPUT RESULTS
# ------------------------

for v in model.variables():
    if v.varValue > 0:
        print(f"{v.name} = {v.varValue}")

print("Total Cost = ", pulp.value(model.objective))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/samanthabritoozuna/anaconda3/envs/tecs6-arm/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/4d9728a5597148f89ada5df7b63c33a4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/4d9728a5597148f89ada5df7b63c33a4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 56 RHS
At line 65 BOUNDS
At line 66 ENDATA
Problem MODEL has 8 rows, 12 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-1) rows, 6 (-6) columns and 18 (-12) elements
0  Obj 0 Primal inf 120 (3)
4  Obj 1600
Optimal - objective value 1600
After Postsolve, objective 1600, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1600 - 4 iterations time 0.002, Presolve 0.00
Option for

# Dataset

In [5]:
import pandas as pd

In [6]:
# HIERARCHICAL OPTIMIZATION ALGORITHM TEMPLATE

import numpy as np
from sklearn.cluster import AgglomerativeClustering
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary, LpStatus

# ----------------------
# Step 1: Input Definitions
# ----------------------
Polygons = list(range(31))  # Polygon IDs from 0 to 30
StorageSites = ['S1', 'S2', 'S3', 'S4']
Customers = ['C1', 'C2', 'C3', 'C4', 'C5']
TimeSlots = ['T1', 'T2', 'T3']

# Sample distances (symmetric matrix for clustering)
DistanceMatrix = np.random.randint(1, 100, size=(31, 31))
DistanceMatrix = (DistanceMatrix + DistanceMatrix.T) / 2

# Demand and capacities (random example)
Demand = {c: {t: np.random.randint(5, 20) for t in TimeSlots} for c in Customers}
Capacity = {s: 100 for s in StorageSites}
DeliveryWindow = {c: TimeSlots for c in Customers}

# ----------------------
# Step 2: Hierarchical Clustering
# ----------------------
def hierarchical_clustering(distance_matrix, num_clusters):
    clustering = AgglomerativeClustering(n_clusters=num_clusters, affinity='precomputed', linkage='average')
    labels = clustering.fit_predict(distance_matrix)
    return {i: labels[i] for i in range(len(labels))}

ClusterAssignment = hierarchical_clustering(DistanceMatrix, num_clusters=5)

# ----------------------
# Step 3: Storage Site Selection
# ----------------------
model = LpProblem("Site_Selection_Problem", LpMinimize)

x = {s: LpVariable(f"open_{s}", cat=LpBinary) for s in StorageSites}
assign = {(s, c): LpVariable(f"assign_{s}_{c}", cat=LpBinary) for s in StorageSites for c in Customers}

open_cost = {s: 100 for s in StorageSites}
serve_cost = {(s, c): np.random.randint(5, 15) for s in StorageSites for c in Customers}

model += lpSum(open_cost[s] * x[s] for s in StorageSites) + \
         lpSum(serve_cost[s, c] * assign[s, c] for s in StorageSites for c in Customers)

for c in Customers:
    model += lpSum(assign[s, c] for s in StorageSites) == 1

for s in StorageSites:
    model += lpSum(Demand[c][t] * assign[s, c] for c in Customers for t in TimeSlots) <= Capacity[s]

for s in StorageSites:
    for c in Customers:
        model += assign[s, c] <= x[s]

model.solve()
print("Storage Site Selection Status:", LpStatus[model.status])

# ----------------------
# Step 4: Delivery Scheduling
# ----------------------
delivery = {(s, c, t): LpVariable(f"delivery_{s}_{c}_{t}", lowBound=0) 
            for s in StorageSites for c in Customers for t in TimeSlots}

schedule_model = LpProblem("Scheduling_Problem", LpMinimize)
delivery_cost = {(s, c, t): np.random.randint(1, 5) for s in StorageSites for c in Customers for t in TimeSlots}

schedule_model += lpSum(delivery_cost[s, c, t] * delivery[s, c, t] 
                        for s in StorageSites for c in Customers for t in TimeSlots)

for c in Customers:
    for t in TimeSlots:
        schedule_model += lpSum(delivery[s, c, t] for s in StorageSites) == Demand[c][t]

for s in StorageSites:
    for t in TimeSlots:
        schedule_model += lpSum(delivery[s, c, t] for c in Customers) <= Capacity[s]

schedule_model.solve()
print("Scheduling Status:", LpStatus[schedule_model.status])


/Users/samanthabritoozuna/anaconda3/envs/tecs6-arm/lib/python3.12/site-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/samanthabritoozuna/anaconda3/envs/tecs6-arm/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/b7642dbe3f824597a4aced06b9c06f91-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/b7642dbe3f824597a4aced06b9c06f91-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 34 COLUMNS
At line 187 RHS
At line 217 BOUNDS
At line 242 ENDATA
Problem MODEL has 29 rows, 24 columns and 80 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 137.195 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0004I processed model has 29 rows, 24 columns (24 integer (24 of which binary)) and 84 elements
Cutoff increment increased from 1e-05 to 0.9999
C

In [7]:
import pulp
from math import ceil
import pandas as pd
import numpy as np
from itertools import product

# ========== Parameters and Data ==========

# Time parameters
days = 30  # Planning horizon (can be adjusted)
work_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
daily_hours = {'Mon': 6, 'Tue': 6, 'Wed': 6, 'Thu': 6, 'Fri': 6, 'Sat': 3}
min_daily_ha = 5  # Minimum 5 hectares per full work day

# Planting parameters
planting_cost_per_plant = 20  # $20 per plant
truck_cost = 4500  # $4500 per truck
truck_capacity = 8000  # plants per truck
storage_capacity = 400  # m²
treatment_area = 100  # m²
plant_footprint = 0.25 * 0.25  # m² per plant (25cm x 25cm)

# Calculate maximum plants that can be stored
max_storage_plants = int(storage_capacity / plant_footprint)
max_treatment_plants = int(treatment_area / plant_footprint)

# Polygons
polygons = range(1, 32)  # 31 polygons
storage_polygon = 18  # Storage is at polygon 18

# Nurseries
nurseries = ['Nursery1', 'Nursery2', 'Nursery3', 'Nursery4']

# Planting density (plants per hectare) - this is an assumption
plants_per_ha = 2000  # Adjust based on your actual density

# ========== Create Optimization Model ==========
model = pulp.LpProblem("Reforestation_Optimization", pulp.LpMinimize)

# ========== Decision Variables ==========

# 1. Ordering variables
orders = pulp.LpVariable.dicts(
    "Order", 
    [(n, d) for n in nurseries for d in range(days)], 
    lowBound=0, 
    upBound=truck_capacity, 
    cat='Integer')

# Whether we order from nursery n on day d (binary)
order_flags = pulp.LpVariable.dicts(
    "OrderFlag",
    [(n, d) for n in nurseries for d in range(days)],
    cat='Binary')

# 2. Storage variables
storage = pulp.LpVariable.dicts(
    "Storage",
    [d for d in range(days + 7)],  # +7 to account for plants staying up to 7 days
    lowBound=0,
    upBound=max_storage_plants,
    cat='Integer')

# 3. Treatment variables
plants_needing_treatment = 0.2  # Assume 20% need treatment - adjust as needed
treatment = pulp.LpVariable.dicts(
    "Treatment",
    [d for d in range(days)],
    lowBound=0,
    upBound=max_treatment_plants,
    cat='Integer')

# 4. Planting variables
# Plants planted in polygon p on day d
planting = pulp.LpVariable.dicts(
    "Planting",
    [(p, d) for p in polygons for d in range(days)],
    lowBound=0,
    cat='Integer')

# 5. Transportation variables
# Truck trips to polygon p on day d
truck_trips = pulp.LpVariable.dicts(
    "TruckTrip",
    [(p, d) for p in polygons for d in range(days)],
    lowBound=0,
    upBound=10,  # Reasonable upper bound
    cat='Integer')

# ========== Constraints ==========

# 1. Ordering constraints
for n in nurseries:
    for d in range(days):
        # Link order flag to actual order (if order > 0, flag must be 1)
        model += orders[(n, d)] <= truck_capacity * order_flags[(n, d)]
        model += orders[(n, d)] >= order_flags[(n, d)]  # If flag=1, order must be at least 1

# 2. Storage constraints
for d in range(days):
    # Plants arrive next day (d+1) if ordered today (d)
    arrivals = pulp.lpSum(orders[(n, d)] for n in nurseries)
    
    # Storage balance equation
    if d == 0:
        model += storage[d] == arrivals  # Initial condition
    else:
        model += storage[d] == storage[d-1] + arrivals - pulp.lpSum(planting[(p, d-1)] for p in polygons)
    
    # Plants can only be planted 3-7 days after arrival
    # This is handled by ensuring plants ordered on day d can only be planted from day d+3 onward
    
    # Storage capacity
    model += storage[d] <= max_storage_plants

# 3. Treatment constraints
for d in range(days):
    # Plants needing treatment (20% of planted)
    model += treatment[d] >= 0.2 * pulp.lpSum(planting[(p, d)] for p in polygons)
    model += treatment[d] <= max_treatment_plants

# 4. Planting constraints
for d in range(days):
    day_type = work_days[d % len(work_days)]
    work_hours = daily_hours[day_type]
    
    # Calculate total plants that can be planted this day based on work hours
    # Assuming a planting rate (plants per hour) - this needs to be estimated
    planting_rate = 500  # plants per hour per crew - adjust based on your data
    max_daily_planting = planting_rate * work_hours
    
    # Total planting constraint
    model += pulp.lpSum(planting[(p, d)] for p in polygons) <= max_daily_planting
    
    # Minimum 5ha per full work day (Mon-Fri)
    if day_type in ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']:
        min_plants = min_daily_ha * plants_per_ha
        model += pulp.lpSum(planting[(p, d)] for p in polygons) >= min_plants

# 5. Transportation constraints
for p in polygons:
    for d in range(days):
        # Plants transported must match plants planted
        model += planting[(p, d)] <= truck_trips[(p, d)] * truck_capacity
        
        # Each truck trip takes time (30 min loading + 30 min unloading + travel time)
        # This would need to be factored into the work hours constraint

# 6. Plant availability constraints
for p in polygons:
    for d in range(days):
        # Plants can only be planted if they're available in storage
        # Considering the 3-7 day window from arrival to planting
        model += planting[(p, d)] <= pulp.lpSum(
            orders[(n, d_prime)] 
            for n in nurseries 
            for d_prime in range(max(0, d-7), max(0, d-2))
            if d_prime + 3 <= d <= d_prime + 7
        )

# ========== Objective Function ==========

# Minimize total costs:
# 1. Ordering costs (truck costs)
ordering_costs = pulp.lpSum(
    order_flags[(n, d)] * truck_cost 
    for n in nurseries 
    for d in range(days)
)

# 2. Planting costs
planting_costs = pulp.lpSum(
    planting[(p, d)] * planting_cost_per_plant 
    for p in polygons 
    for d in range(days)
)

# 3. Maximize survival rate (simplified as minimizing time in storage)
# Plants in storage more days have lower survival - we'll minimize storage days
storage_days = pulp.lpSum(storage[d] for d in range(days))

# Combined objective (weighted)
model += ordering_costs + planting_costs + 0.1 * storage_days

# ========== Solve the Model ==========
model.solve()

# ========== Results Analysis ==========
if pulp.LpStatus[model.status] == 'Optimal':
    print("Optimal solution found!")
    
    # Extract order schedule
    order_schedule = []
    for n in nurseries:
        for d in range(days):
            if pulp.value(order_flags[(n, d)]) > 0.5:
                order_schedule.append({
                    'Nursery': n,
                    'Order Day': d,
                    'Plants': pulp.value(orders[(n, d)]),
                    'Arrival Day': d+1
                })
    
    order_df = pd.DataFrame(order_schedule)
    print("\nOrder Schedule:")
    print(order_df)
    
    # Extract planting schedule
    planting_schedule = []
    for p in polygons:
        for d in range(days):
            if pulp.value(planting[(p, d)]) > 0:
                planting_schedule.append({
                    'Polygon': p,
                    'Planting Day': d,
                    'Plants': pulp.value(planting[(p, d)]),
                    'Hectares': pulp.value(planting[(p, d)]) / plants_per_ha
                })
    
    planting_df = pd.DataFrame(planting_schedule)
    print("\nPlanting Schedule:")
    print(planting_df.groupby('Planting Day').sum())
    
    # Calculate costs
    total_cost = pulp.value(ordering_costs + planting_costs)
    print(f"\nTotal Cost: ${total_cost:,.2f}")
    
else:
    print("No optimal solution found. Status:", pulp.LpStatus[model.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/samanthabritoozuna/anaconda3/envs/tecs6-arm/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/83a9ed3221b2476397f8f8d565c5c52a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/83a9ed3221b2476397f8f8d565c5c52a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2280 COLUMNS
At line 30254 RHS
At line 32530 BOUNDS
At line 34691 ENDATA
Problem MODEL has 2275 rows, 2160 columns and 22573 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.03

No optimal solution found. Status: Infeasible
